In [2]:
from tweedledum.bool_function_compiler import BitVec, BoolFunction

def f(v0, v1, v2, v3 : BitVec(1)) -> BitVec(1):
    return (v0 != v1) and (v2 != v3) and (v0 != v2) and (v1 != v3)

oracle_func = BoolFunction(f)

In [3]:
# Using l337 programming skills:
for i in range(2**(4)):
    v = BitVec(4, i)
    result = f(v[0], v[1], v[2], v[3])
    if result:
        print(v[0], v[1], v[2], v[3])

0 1 1 0
1 0 0 1


## Configurations

In [4]:
syntehsis_method = 'pkrm' # (pkrm, xag)

# Optimization options
use_barenco_decomp = True

## Oracle

In [5]:
from tweedledum.synthesis import xag_synth, pkrm_synth

def use_pkrm(bool_function):
    return pkrm_synth(bool_function.truth_table(output_bit=0))

def use_xag(bool_function):
    from tweedledum.passes import parity_decomp
    circuit = xag_synth(bool_function.logic_network())
    return parity_decomp(circuit)

In [6]:
oracle_circuit = use_pkrm(oracle_func) if syntehsis_method == 'pkrm' else use_xag(oracle_func)
if use_barenco_decomp:
    from tweedledum.passes import barenco_decomp
    oracle_circuit = barenco_decomp(oracle_circuit, {'max_qubits' : 16})

print(oracle_circuit)
print(f"Number of qubits: {oracle_circuit.num_qubits()}")
print(f"Number of instructions: {len(oracle_circuit)}")

       ╭────╮                 ╭────╮      ╭────╮     ╭────╮      
__a6 : ┤ rx ├──●──────────●───┤ rx ├──────┤ rx ├──●──┤ rx ├──────
       ╰─┬──╯╭─┴──╮     ╭─┴──╮╰─┬──╯╭────╮╰─┬──╯  │  ╰─┬──╯╭────╮
__a5 : ──┼───┤ rx ├──●──┤ rx ├──┼───┤ rx ├──●─────┼────●───┤ rx ├
         │   ╰─┬──╯╭─┴─╮╰─┬──╯  │   ╰─┬──╯  │   ╭─┴─╮  │   ╰─┬──╯
__q4 : ──┼─────┼───┤ x ├──┼─────┼─────┼─────┼───┤ x ├──┼─────┼───
         │     │   ╰─┬─╯  │     │     │     │   ╰─┬─╯  │     │   
__q3 : ──┼─────┼─────●────┼─────┼─────┼─────┼─────◯────┼─────┼───
         │     │          │     │     │     │          │     │   
__q2 : ──┼─────◯──────────◯─────┼─────┼─────●──────────●─────┼───
         │                      │     │                      │   
__q1 : ──◯──────────────────────◯─────●──────────────────────●───
         │                      │     │                      │   
__q0 : ──●──────────────────────●─────◯──────────────────────◯───
                                                                 

Number of

In [7]:
from tweedledum.ir import Circuit
from tweedledum.operators import H, Measure, X, Z
from tweedledum.passes import barenco_decomp

# Initialize
circuit = Circuit()
qubits = [circuit.create_qubit() for i in range(oracle_circuit.num_qubits())]
cbits = [circuit.create_cbit() for i in range(4)]

circuit.apply_operator(X(), [qubits[4]])
for i in range(5):
    circuit.apply_operator(H(), [qubits[i]])

for i in range(2):
    circuit.append(oracle_circuit, qubits[0:oracle_circuit.num_qubits()], [])
    for i in range(4):
        circuit.apply_operator(H(), [qubits[i]])
        circuit.apply_operator(X(), [qubits[i]])
    circuit.apply_operator(Z(), qubits[0:4])
    for i in range(4):
        circuit.apply_operator(X(), [qubits[i]])
        circuit.apply_operator(H(), [qubits[i]])

for i in range(circuit.num_cbits()):
    circuit.apply_operator(Measure(), [qubits[i]], [cbits[i]])
    
print(circuit)
circuit = barenco_decomp(circuit, {'max_qubits' : 32})

                 ╭────╮                 ╭────╮      ╭────╮     ╭────╮      »
__q6 : ──────────┤ rx ├──●──────────●───┤ rx ├──────┤ rx ├──●──┤ rx ├──────»
                 ╰─┬──╯╭─┴──╮     ╭─┴──╮╰─┬──╯╭────╮╰─┬──╯  │  ╰─┬──╯╭────╮»
__q5 : ────────────┼───┤ rx ├──●──┤ rx ├──┼───┤ rx ├──●─────┼────●───┤ rx ├»
       ╭───╮╭───╮  │   ╰─┬──╯╭─┴─╮╰─┬──╯  │   ╰─┬──╯  │   ╭─┴─╮  │   ╰─┬──╯»
__q4 : ┤ x ├┤ h ├──┼─────┼───┤ x ├──┼─────┼─────┼─────┼───┤ x ├──┼─────┼───»
       ├───┤╰───╯  │     │   ╰─┬─╯  │     │     │     │   ╰─┬─╯  │     │   »
__q3 : ┤ h ├───────┼─────┼─────●────┼─────┼─────┼─────┼─────◯────┼─────┼───»
       ├───┤       │     │          │     │     │     │          │     │   »
__q2 : ┤ h ├───────┼─────◯──────────◯─────┼─────┼─────●──────────●─────┼───»
       ├───┤       │                      │     │                      │   »
__q1 : ┤ h ├───────◯──────────────────────◯─────●──────────────────────●───»
       ├───┤       │                      │     │                      │   »

In [8]:
from tweedledum.converters_qiskit import tweedledum_to_qiskit_qc

qiskit_circuit = tweedledum_to_qiskit_qc(circuit)
print(qiskit_circuit)

     ┌───┐                                                                  »
q_0: ┤ H ├─────────■────────────────────────────────■─────────o─────────────»
     ├───┤         │                                │         │             »
q_1: ┤ H ├─────────o────────────────────────────────o─────────■─────────────»
     ├───┤         │                                │         │             »
q_2: ┤ H ├─────────┼────────o─────────────o─────────┼─────────┼────────■────»
     ├───┤         │        │             │         │         │        │    »
q_3: ┤ H ├─────────┼────────┼──────■──────┼─────────┼─────────┼────────┼────»
     ├───┤┌───┐    │        │    ┌─┴─┐    │         │         │        │    »
q_4: ┤ X ├┤ H ├────┼────────┼────┤ X ├────┼─────────┼─────────┼────────┼────»
     └───┘└───┘    │    ┌───┴───┐└─┬─┘┌───┴────┐    │     ┌───┴───┐    │    »
q_5: ──────────────┼────┤ RX(π) ├──■──┤ RX(-π) ├────┼─────┤ RX(π) ├────■────»
               ┌───┴───┐└───┬───┘     └───┬────┘┌───┴────┐└─────

In [9]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
qiskit_circuit = pm.run(qiskit_circuit) 

In [10]:
import qiskit
from qiskit.providers.aer import QasmSimulator

# Construct an ideal simulator
sim = QasmSimulator()

# Perform an ideal simulation
result_ideal = qiskit.execute(qiskit_circuit, sim).result()
counts_ideal = result_ideal.get_counts(0)
print('Counts(ideal):', counts_ideal)

Counts(ideal): {'0000': 2, '0001': 3, '0010': 2, '0011': 2, '0100': 4, '0101': 4, '0110': 474, '0111': 4, '1000': 9, '1001': 491, '1010': 2, '1011': 9, '1100': 4, '1101': 3, '1110': 4, '1111': 7}
